In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("../assets/cran_packages_dropped.csv")
df.head()

,Package,Version,Depends,Imports,LinkingTo,Suggests,License,MD5sum,NeedsCompilation,Repository,count
0,A3,1.0.0,"R (>= 2.15.0), xtable, pbapply",NaN,NaN,"randomForest, e1071",GPL (>= 2),027ebdd8affce8f0effaecfcd5f5ade2,no,https://cran.rstudio.com/src/contrib,116853
1,AalenJohansen,1.0,NaN,NaN,NaN,"knitr, rmarkdown",GPL (>= 2),d7eb2a6275daa6af43bf8a980398b312,no,https://cran.rstudio.com/src/contrib,3356
2,AATtools,0.0.2,R (>= 3.6.0),"magrittr, dplyr, doParallel, foreach",NaN,NaN,GPL-3,bc59207786e9bc49167fd7d8af246b1c,no,https://cran.rstudio.com/src/contrib,20756
3,ABACUS,1.0.0,R (>= 3.1.0),"ggplot2 (>= 3.1.0), shiny (>= 1.3.1),",NaN,"rmarkdown (>= 1.13), knitr (>= 1.22)",GPL-3,50c54c4da09307cb95a70aaaa54b9fbd,no,https://cran.rstudio.com/src/contrib,27700
4,abasequence,0.1.0,NaN,NaN,NaN,NaN,GPL-3,1392d909eb0f65be94fd4160a371ae21,no,https://cran.rstudio.com/src/contrib,1856


In [3]:
df = df[["Package", "Imports", "Suggests"]]
df.head()

,Package,Imports,Suggests
0,A3,NaN,"randomForest, e1071"
1,AalenJohansen,NaN,"knitr, rmarkdown"
2,AATtools,"magrittr, dplyr, doParallel, foreach",NaN
3,ABACUS,"ggplot2 (>= 3.1.0), shiny (>= 1.3.1),","rmarkdown (>= 1.13), knitr (>= 1.22)"
4,abasequence,NaN,NaN


In [4]:
df.tail()

,Package,Imports,Suggests
20097,zTree,plyr (>= 1.0),NaN
20098,ztype,"magrittr, rvest, stats, stringr, utils, dplyr,...",NaN
20099,ZVCV,"Rcpp (>= 0.11.0), glmnet, abind, mvtnorm, stat...","partitions, ggplot2, ggthemes"
20100,zyp,NaN,NaN
20101,zzlite,"httr, jsonlite","httptest, roxygen2, testthat, knitr, rmarkdown"


In [5]:
def rem_brackets(string):
    if isinstance(string, str):
        return re.sub(r'\([^()]*\)', '', string)
    return string


df[['Imports', 'Suggests']] = df[['Imports', 'Suggests']].map(rem_brackets)

In [6]:
packs = {}
for _, row in df.iterrows():
    package_name = row['Package']
    imports = row['Imports'] if pd.notnull(row['Imports']) else []
    suggests = row['Suggests'] if pd.notnull(row['Suggests']) else []
    packs[package_name.lower()] = {'imports': imports, 'suggests': suggests}

packs["abacus"]

{'imports': 'ggplot2 , shiny ,', 'suggests': 'rmarkdown , knitr '}

In [7]:
len(packs)

20102

In [8]:
known = set()

for package in packs:
    known.add(package.lower())

len(known)

20102

In [9]:
for package in packs:
    if packs[package]['imports']:
        imports = set([ele.strip().lower() for ele in (packs[package]["imports"]).split(",") if ele.strip().lower() in known])

        if "" in imports:
            imports.remove("")

    if packs[package]['suggests']:
        suggests = set([ele.strip().lower() for ele in (packs[package]["suggests"]).split(",") if ele.strip().lower() in known])

        if "" in suggests:
            suggests.remove("")
    
    packs[package]["imports"] = list(imports)
    packs[package]["suggests"] = list(suggests)

In [10]:
packs["abacus"]

{'imports': ['shiny', 'ggplot2'], 'suggests': ['knitr', 'rmarkdown']}

In [11]:
totality = set()
known = set()

for package in packs:
    totality.add(package.lower())
    known.add(package.lower())
    
    totality.update(packs[package]["imports"])
    totality.update(packs[package]["suggests"])

len(totality)

20114

In [12]:
unknown = totality.difference(known)

In [13]:
for package in packs:
    if packs[package]['imports']:
        imports = [ele for ele in (packs[package]["imports"]) if ele in known]

    if packs[package]['suggests']:
        suggests = [ele for ele in (packs[package]["suggests"]) if ele in known]

    packs[package]["imports"] = list(imports)
    packs[package]["suggests"] = list(suggests)

In [14]:
totality = set()
known = set()

for package in packs:
    totality.add(package.lower())
    known.add(package.lower())
    
    totality.update(packs[package]["imports"])
    totality.update(packs[package]["suggests"])

len(totality)

20102

In [15]:
index = {}
label = {}

i = 1

for package in packs:
    index[package.lower()] = i
    label[i] = package.lower()
    i += 1

print(index["abacus"])
label[4]

4


'abacus'

In [16]:
dependecy = {}

for package in packs:
    dependecy[index[package.lower()]] = {}

    strong = [index[ele.lower()] for ele in packs[package]["imports"]]
    weak = [index[ele.lower()] for ele in packs[package]["suggests"]]

    dependecy[index[package.lower()]]["strong"] = strong
    dependecy[index[package.lower()]]["weak"] = weak

dependecy[4]

{'strong': [16523, 6644], 'weak': [8780, 15103]}

In [17]:
label[16523]

'shiny'

In [18]:
import json

json.dump(dependecy, open("./dependecy.json", "w"))
json.dump(index, open("./indices.json", "w"))
json.dump(label, open("./labels.json", "w"))